In [1]:
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [2]:
log_train_df= log_train
train_df = train

In [3]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')

# Fill missing values for IDs that did not have event data


train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN

# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'median' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

In [5]:
import optuna
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Assuming X_resampled and y_resampled are your features and target after resampling

def objective(trial):
    # Splitting data for hyperparameter optimization
    X_train, X_valid, y_train, y_valid = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    # Defining hyperparameter search space
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'dart',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
    }

    # Training the model with the trial's parameters
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
    preds = gbm.predict(X_valid)
    # Calculating log loss for the validation set
    loss = log_loss(y_valid, preds)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) # You can adjust the number of trials

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-03-27 01:48:36,064] A new study created in memory with name: no-name-8f60d174-b0b9-4524-a4b0-0e9b5cabf22f
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.335667
[200]	valid_0's binary_logloss: 0.280333
[300]	valid_0's binary_logloss: 0.246941
[400]	valid_0's binary_logloss: 0.231763
[500]	valid_0's binary_logloss: 0.223697
[600]	valid_0's binary_logloss: 0.222165
[700]	valid_0's binary_logloss: 0.218853
[800]	valid_0's binary_logloss: 0.216973
[900]	valid_0's binary_logloss: 0.214479
[1000]	valid_0's binary_logloss: 0.213249
[1100]	valid_0's binary_logloss: 0.211853
[1200]	valid_0's binary_logloss: 0.21112
[1300]	valid_0's binary_logloss: 0.210283
[1400]	valid_0's binary_logloss: 0.208945
[1500]	valid_0's binary_logloss: 0.20885
[1600]	valid_0's binary_logloss: 0.207951
[1700]	valid_0's binary_logloss: 0.207119
[1800]	valid_0's binary_logloss: 0.206782
[1900]	valid_0's binary_logloss: 0.206486
[2000]	valid_0's binary_logloss: 0.206219


[I 2024-03-27 01:51:19,302] Trial 0 finished with value: 0.20621899492911272 and parameters: {'learning_rate': 0.06685776629525489, 'num_leaves': 299, 'max_depth': 3, 'min_child_samples': 93, 'subsample': 0.7718732198456097, 'colsample_bytree': 0.8532674879658929, 'reg_alpha': 0.02429488276859937, 'reg_lambda': 0.9048692747379464}. Best is trial 0 with value: 0.20621899492911272.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.207867
[200]	valid_0's binary_logloss: 0.208798
[300]	valid_0's binary_logloss: 0.213575
[400]	valid_0's binary_logloss: 0.219681
[500]	valid_0's binary_logloss: 0.228652
[600]	valid_0's binary_logloss: 0.231728
[700]	valid_0's binary_logloss: 0.238729
[800]	valid_0's binary_logloss: 0.243222
[900]	valid_0's binary_logloss: 0.250087
[1000]	valid_0's binary_logloss: 0.253633
[1100]	valid_0's binary_logloss: 0.258698
[1200]	valid_0's binary_logloss: 0.260601
[1300]	valid_0's binary_logloss: 0.262949
[1400]	valid_0's binary_logloss: 0.267912
[1500]	valid_0's binary_logloss: 0.267881
[1600]	valid_0's binary_logloss: 0.271585
[1700]	valid_0's binary_logloss: 0.27503
[1800]	valid_0's binary_logloss: 0.27609
[1900]	valid_0's binary_logloss: 0.278249
[2000]	valid_0's binary_logloss: 0.278805


[I 2024-03-27 02:17:36,690] Trial 1 finished with value: 0.27880451935491135 and parameters: {'learning_rate': 0.25473287735401173, 'num_leaves': 195, 'max_depth': 15, 'min_child_samples': 82, 'subsample': 0.9166312279129013, 'colsample_bytree': 0.599540417630458, 'reg_alpha': 0.6515401056720604, 'reg_lambda': 0.7640898815659358}. Best is trial 0 with value: 0.20621899492911272.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


...

[100]	valid_0's binary_logloss: 0.398594
[200]	valid_0's binary_logloss: 0.335192
[300]	valid_0's binary_logloss: 0.289386
[400]	valid_0's binary_logloss: 0.260905
[500]	valid_0's binary_logloss: 0.241634
[600]	valid_0's binary_logloss: 0.23735
[700]	valid_0's binary_logloss: 0.230317
[800]	valid_0's binary_logloss: 0.227211
[900]	valid_0's binary_logloss: 0.222953
[1000]	valid_0's binary_logloss: 0.220904
[1100]	valid_0's binary_logloss: 0.21786
[1200]	valid_0's binary_logloss: 0.217537
[1300]	valid_0's binary_logloss: 0.216104
[1400]	valid_0's binary_logloss: 0.213862
[1500]	valid_0's binary_logloss: 0.21366
[1600]	valid_0's binary_logloss: 0.21206
[1700]	valid_0's binary_logloss: 0.210851
[1800]	valid_0's binary_logloss: 0.210401
[1900]	valid_0's binary_logloss: 0.209523
[2000]	valid_0's binary_logloss: 0.209065


[I 2024-03-27 04:26:14,243] Trial 11 finished with value: 0.20906499645536947 and parameters: {'learning_rate': 0.03223094927436972, 'num_leaves': 214, 'max_depth': 4, 'min_child_samples': 81, 'subsample': 0.7056843499884327, 'colsample_bytree': 0.6489570809023841, 'reg_alpha': 0.411696623198278, 'reg_lambda': 0.676454069549168}. Best is trial 0 with value: 0.20621899492911272.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.280738
[200]	valid_0's binary_logloss: 0.241718
[300]	valid_0's binary_logloss: 0.221356
[400]	valid_0's binary_logloss: 0.213761
[500]	valid_0's binary_logloss: 0.209119
[600]	valid_0's binary_logloss: 0.20921


[W 2024-03-27 04:27:52,969] Trial 12 failed with parameters: {'learning_rate': 0.06007314176405205, 'num_leaves': 191, 'max_depth': 6, 'min_child_samples': 99, 'subsample': 0.553990603483238, 'colsample_bytree': 0.7122279280052972, 'reg_alpha': 0.34316841386802444, 'reg_lambda': 0.9982545064015643} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/w5/7276v8b14pj5_4lrc2r9xkvr0000gn/T/ipykernel_7437/1606589854.py", line 29, in objective
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py", l

KeyboardInterrupt: 